**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

---

Mentoría #13 - Cómo hacer un Clasificador de Pliegos todoterreno (y de otros tipos de textos) usando NLP

---
Integrantes Grupo [1|2]:
*   Franco Manini: francoamaninig@gmail.com
*   Santiago Rosa: santiago.rosa@mi.unc.edu.ar
*   Gonza
*   Sofia
---

Edición 2023


# Introducción

En esta Mentoría, trabajaremos con un conjunto de datos que comprende aproximadamente 100.000 pliegos y licitaciones de diversos organismos nacionales, tanto públicos como privados. Estos datos se obtuvieron de un sistema/servicio diseñado para monitorear oportunidades de negocios, capturar la información en una base de datos, normalizarla y, posteriormente, clasificarla para informar a los usuarios según sus áreas de interés. Los usuarios de este sistema reciben alertas automáticas cada vez que se publica una oportunidad comercial que coincide con su perfil.

## Desafío

Como parte del proceso de clasificación, los pliegos se etiquetan utilizando principalmente reglas estáticas, como palabras clave, lo cual deja margen para optimizaciones. El título de cada pliego y principalmente la descripción de los objetos que se están licitando son campos de tipo texto y escritos por personas, por lo que naturalmente presentan ambigüedades y características propias del lenguaje que llevan a que un enfoque rígido y estático de clasificación, como el actual, resulte limitado y poco eficiente.

El Desafío es utilizar las técnicas de Aprendizaje Automático, logrando así un "Clasificador" que utilice técnicas de Procesamiento de Lenguaje Natural (NLP) para clasificar de manera más eficiente en qué rubro o categoría se encuentra un pliego, basándose en su texto descriptivo y otros campos relacionados.

## Interés General

Más allá de la aplicación específica en este conjunto de datos, este problema, al estar vinculado al Procesamiento de Lenguaje Natural (PLN) y la clasificación, tiene la ventaja de poder ser utilizado posteriormente para otros tipos de contenido. De esta manera, el clasificador desarrollado podría aplicarse para categorizar libros, noticias, textos, tweets, publicaciones, artículos, etc.

## Descripción del dataset

A continuación se enumeran las diferentes variables del dataset, así como una breve descripción de su significado:

- **id**: Clave única y primaria autoincremental de la tabla;
- **cargado**: Fecha de carga del pliego;
- **idexterno**: Id del pliego en la fuente;
- **referencia**: Campo auxiliar obtenido de la fuente;
- **objeto**: Campo principal, descripción del producto o servicio objeto de la licitación;
- **rubro**: Campo de categorización disponible en la fuente. No siempre está disponible;
- **agencia**: Empresa o Ente que lanza la licitación;
- **apertura**: Fecha de apertura del pliego (vencimiento para presentarse al pliego);
- **subrubro**: Subcategoría del pliego (también obtenido desde la fuente);
- **pais**: País donde se lanza la licitación;
- **observaciones**: Campo auxiliar donde se guardan datos extra que puede variar según la fuente;
- **monto**: Monto del pliego, no siempre está publicado;
- **divisaSimboloISO**: Moneda en la que se especifica el pliego;
- **visible**: campo binario que determina si el pliego va a ser visualizado por el sistema (True/False);
- **categoría**: Tipo de pliego, categorización entre diversos tipos de pliegos (Compra Directa, Licitación Simple, Subasta, etc.);
- **fuente**: Fuente de donde se obtuvo la licitación.

# Trabajo Práctico #1

Como primer trabajo práctico se requiere hacer un análisis exploratorio del dataset, analizando distribuciones, dispersión de las variables, naturaleza de los datos alojados en cada variable, etc. Es una primera aproximación a los datos, por lo que, como todo trabajo de exploración, requiere principalmente de técnicas de visualización, agrupación y lectura de las distintas variables.

A continuación se enumeran los ejercicios propuestos como guía:

## Ejercicio 1: Descripción

Hacer una descripción propia del dataset a modo de resumen, resaltando las variables que se consideran más importantes.

Describir las principales variables según las características de los datos, por ejemplo, explorar la distribución de los valores, la característica de los mismos, explorar el conjunto para evidenciar si existen valores faltantes.

De preferencia, utilizar visualizaciones en esta etapa de exploración

## Ejercicio 2: Exploración

Analizar ahora con mayor detalle las variables importantes dentro del conjunto:

*   Hay variables numéricas?
*   Son del tipo contínuo o discreta
*   Hay variables categóricas?
*   Hay variables "derivadas" de otras variables
*   En las variables de tipo texto, es homogénea la longitud de los datos?
*   Hay registros con mayor longitud que otros?
*   Representará esto un problema a tratar más adelante?
*   Existen desbalances en la distribución o son los datos homogéneos?
*   Existen datos faltantes?

Nuevamente, utilizar las preguntas descriptas como guía para la exploración y hacer gráficas y visualizaciones como forma práctica de evidenciar lo observado.

## Ejercicio 3: Análisis

A partir de los datos y de la exploración realizada, qué podemos anticipar en relación a las variables *rubro* y *subrubro*?

Están siempre presentes? Representan algún tipo de desbalanceo?

Existen pliegos con más de una categorización en la fuente de origen? Qué distribución hay en ese caso?

Nota: Aquí ya nos topamos con una de las problemáticas del dataset. Es esta categorización en origen "confiable"? Qué posturas podemos tomar frente a estas etiquetas originales?

Describir en forma de texto la postura del Grupo respecto a estas variables y a posibles estrategias para abordarlas en adelante. Este será un factor muy importante cuando comencemos a preparar el dataset para alimentar los modelos de clasificación.

# Desarrollo

In [ ]:
# Link al dataset: https://drive.google.com/file/d/1jo9cZNHAPpsTosW-2JA4CLo4SaFCHfIs/view?usp=drive_link

Veamos la pinta del dataset:

In [ ]:
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np

pd.set_option('display.max_rows', None)

df = pd.read_csv("./Mentoria_Dataset_0.csv" , encoding='latin-1')

df.head()

#print(df['fuente'].unique())
#print(df['agencia'].unique())

#print(len(df))


detalles del dataset: 16 columnas, 104999 filas.

# columnas que consideramos relevantes: 

* objeto
* rubro
* agencia
* fuente
* apertura
* monto
* categoria
* idexterno


# datos redundantes: 

* idexterno: es una variable compuesta (referencia + subrubro)
* pais: son todos los pliegos de Argentina
* fecha de carga: irrelevante, importa más la comparada con fecha del pliego
* divisaSimboloISO: incompleta, se asume que todo está en pesos
* subrubro: incompleta
* observaciones: muy incompleta
* indice: es el mismo index de pandas



# Datos faltantes

Nos fijamos usando missingno los datos faltantes. Vemos que no todas las entradas tienen un valor en 'monto'.


Consigna: explorar la distribución de los valores, la característica de los mismos, explorar el conjunto para evidenciar si existen valores faltantes.

In [ ]:
rel_col = ["objeto",'rubro','agencia',"fuente",'apertura','monto','categoria','idexterno']

fig, axs = plt.subplots(figsize=(6, 5))
msno.bar(df[rel_col], sort="ascending", fontsize=12, color="tab:green", ax=axs)
#axs.set_ylim(0.8, 1)
plt.show()

Ahora nos fijamos en la descripción de las principales variables elegidas:



In [ ]:
print(df['rubro'].unique())
print('rubro: ',len(df['rubro'].unique()))
print('objeto: ',len(df['objeto'].unique()))
print('agencia: ',len(df['agencia'].unique()))
print('fuente: ',len(df['fuente'].unique()))
print('apertura: ',len(df['apertura'].unique()))
print('monto: ',len(df['monto'].unique()))
print('categoria: ',len(df['categoria'].unique()))
print('idexterno: ',len(df['idexterno'].unique()))


Ahora nos fijamos en la cantidad de entradas que tienen las columnas fuente, categoria y visible:

## duda: qué es cada categoría?

## proposición: 
* filtrar las variables 'categoría' y 'visible', dado que son variables binarias muy sesgadas (quedarnos en ambos casos con 1).
* Trabajar la variable 'monto', que está bastante fea
* Hacer una columna nueva con la longitud de la string de la variable 'objeto', y hacer un histograma
* filtrar apertura por mes o año, para disminuir la cantidad de entradas distintas
* ver que las fechas de cargado estén llenas de la misma forma
* ver que las entradas de 'agencia' estén bien escritas y que no haya repetidas pero mal escritas
* damos por hecho que todo es de argentina

In [ ]:
frecuencia_fuente = df['fuente'].value_counts()
plt.bar(frecuencia_fuente.index, frecuencia_fuente.values)
plt.xticks(rotation=90)
plt.xlabel('Fuente')
plt.ylabel('Frecuencia')
plt.title('Grafico de barras de las fuentes')
plt.show()


En el siguiente análisis, vemos que la variable "categoría" es binaria, y la gran cantidad de entradas es 1. Descartamos esta variable para futuros modelos.

In [ ]:
print(df['categoria'].unique())

frecuencia_categoria = df['categoria'].value_counts()
plt.bar(frecuencia_categoria.index, frecuencia_categoria.values)
plt.xticks([1,2],  ['1','2'])
plt.xlabel('Categorias')
plt.ylabel('Frecuencia')
plt.title('Grafico de barras de las categorías')
plt.legend()
plt.show()

#tiro los que tienen categoria 2
df1 = df[df['categoria']==1]


Mismo análisis que para la variable "categoría" pero para la variable "visibilidad". Descartamos esta variable dado que predomina claramente una entrada (la visible). 

In [ ]:
frecuencia_vis = df1['visible'].value_counts()
plt.bar(frecuencia_vis.index, frecuencia_vis.values)
plt.xticks([0,1],  ['0','1'])
plt.xlabel('Visible')
plt.ylabel('Frecuencia')
plt.title('Grafico de barras de la visibilidad')
plt.show()

#tiro los que tienen visibilidad 0
df2 = df1[df1['categoria']==1]


Analizamos ahora la distribución de fechas de inicio de los pliegos. Hay cuatro fechas conflictivas, las descarto.

Tiro las horas para ver si se agiliza el cálculo del histograma e una primera instancia. Además, siguen habiendo o fechas muy viejas o fechas futuras que no tienen. Tiro los pliegos que tienen fechas anteriores a 2015 (ya que los más viejos no tienen peso en el histograma) y los que tienen fecha después de la fecha de hoy. 

In [ ]:
today_date = pd.Timestamp.today().date()
min_date = pd.to_datetime('13/12/2015')
df3 = df2.copy()
df3.reset_index(inplace=True)  #reseteo indices, es util cuando indice=nro de fila

#fechas malas que encontramos:
fecha_mala1 = df3['apertura'][62]
fecha_mala2 = df3['apertura'][2025]
fecha_mala3 = df3['apertura'][11907]
# fecha_mala4 = df3['apertura'][12375]


print(fecha_mala1)
print(fecha_mala2)
print(fecha_mala3)

# #remuevo las fechas malas
df3 = df3.drop(df3[df3['apertura'] == fecha_mala1].index)
df3 = df3.drop(df3[df3['apertura'] == fecha_mala2].index)
df3 = df3.drop(df3[df3['apertura'] == fecha_mala3].index)

#convierto a formato datetime para hacer un histograma
df3['apertura'] = pd.to_datetime(df3['apertura'],format='%d/%m/%Y %H:%M')
df3['apertura'] = df3['apertura'].dt.tz_localize(None)
df3['apertura'] = df3['apertura'].dt.date

#histograma con outliers
plt.hist(df3['apertura'], edgecolor='black')
plt.xticks(rotation=90)
plt.xlabel('fecha de inicio')
plt.ylabel('Frecuencia')
plt.title('Histograma de inicio de pliegos - sin filtrar')
plt.show()

#remuevo outliers
df3 = df3.drop(df3[df3['apertura'] > today_date].index)
df3 = df3.drop(df3[df3['apertura'] < min_date].index)

#histograma sin outliers
plt.hist(df3['apertura'], edgecolor='black')
plt.xticks(rotation=90)
plt.xlabel('fecha de inicio')
plt.ylabel('Frecuencia')
plt.title('Histograma de inicio de pliegos - filtrado')
plt.show()

Ahora analizamos la columna 'objeto'. Es una columna de strings donde se especifica el detalle del pliego. Veamos la distribución de cantidad de caracteres:

In [ ]:
df3['longitud'] = df3['objeto'].str.len()

Q1=df3['longitud'].quantile(0.25)
Q2=df3['longitud'].quantile(0.50)
Q3=df3['longitud'].quantile(0.75)
max=df3['longitud'].max()
min=df3['longitud'].min()
mean=df3['longitud'].mean()
print('Media:', mean)
print('Max:', max)
print('Min:',min)
print('Mediana:',Q2)
print('Q1:',Q1)
print('Q2:',Q2)
print('Q3:',Q3)

plt.hist(df3['longitud'], bins=30)  
plt.xlabel('Longitud de los strings')
plt.ylabel('Frecuencia')
plt.title('Histograma de longitud de strings')
plt.xlim(0,400) #Corto el histograma en 400, aunque llega a 1000 pero son muy pocos casos
plt.show()

Ahora analicemos la columna de montos. Vemos que hay muchos pliegos que se hicieron por muy poca cantidad de dinero, otros donde no hay datos, y otros pliegos gratuitos.

In [ ]:
# Trabajo con la columna de montos
df4 = df3.copy()

valores=df4['monto'].unique()
frecuencia_monto = df4['monto'].value_counts()
print(frecuencia_monto)

Hay muchos valores que son iguales pero tienen distinta cantidad de espacios, por lo que removemos todos los espacios. Algunos pliegos están en dólares así que los conviertimos a pesos, usando el valor del dólar oficial. A los pliegos con valor '0' los pasamos a 0 flotante, y \N a np.nan.

In [ ]:
#precio del dolar oficial
dolar_value = 255

df4 = df3.copy()

df4['monto'] = df4['monto'].replace(r'\N', np.nan)
df4['monto'] = df4['monto'].replace('0', 0)

#remuevo las strings de espacios, pesos y dolares donde las haya. Cambio la representacion de string a float
for monto, i in zip(df4['monto'],df4.index):
    if " " in str(monto):
        df4['monto'][i] = str(df4['monto'][i]).replace(' ','')
    if "$" in str(monto):
        df4['monto'][i] = str(df4['monto'][i]).replace('$','')
    if "." in str(monto):
        df4['monto'][i] = str(df4['monto'][i]).replace('.','')
    if "," in str(monto):
        df4['monto'][i] = str(df4['monto'][i]).replace(',','.')    
    if "U$S" in str(monto):
        df4['monto'][i] = str(df4['monto'][i]).replace('US','')
        #convierto a pesos y reemplazo la entrada
        df4['monto'][i] = float(df4['monto'][i]) * dolar_value  
    df4['monto'][i] = float(df4['monto'][i]) 
        


Veamos ahora la distribución de montos en un histograma. El boxplot no da mucha información dado que la escala es muy grande y el promedio del monto del pliego es pequeño en esta escala.

Por otro lado, un histograma con bineado logarítmico sí deja ver la distribución de montos.

Intuyo que tenemos un crecimiento exponencial de la frecuencia desde el monto 0 hasta cerca del valor medio, y un decrecimiento exponencial desde la media hacia el pliego máximo. Esto es lo que interpreto del gráfico, no tuve tiempo de hacer los ajustes. 

In [ ]:
import seaborn as sns

sns.boxplot(x=df4[df4['monto']>0]['monto'])
plt.show()

bins = np.logspace(0, 10,base=10,num=100)

plt.hist(df4[df4['monto']>0]['monto'], bins=bins)  
plt.xlabel('monto')
plt.ylabel('Frecuencia')
plt.title('Histograma de montos de los pliegos')
#plt.ylim(0,1400)
plt.xscale("log")
plt.yscale("log")
plt.show()